
# Feature Engineering

Create new features based on time and location, such as trip duration, day of the week, hour of the day, and whether the trip occurred during a holiday or special event.


In [17]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import haversine as hs
from datetime import datetime
import folium
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os
import pandas as pd
import requests
from datetime import datetime

os.chdir("/home/tema/10X/Week8/Logistic optimization/data")
os.getcwd()

'/home/tema/10X/Week8/Logistic optimization/data'

In [3]:
delivery_orders = pd.read_csv('delivery_orders.csv')

delivery_orders.head()

,id,order_id,driver_id,driver_action,lat,lng
0,1,392001,243828,accepted,6.602207,3.270465
1,2,392001,243588,rejected,6.592097,3.287445
2,3,392001,243830,rejected,6.596133,3.281784
3,4,392001,243539,rejected,6.596142,3.280526
4,5,392001,171653,rejected,6.609232,3.288800


In [11]:
completed_orders = pd.read_csv('completed_orders.csv')
completed_orders.head(2)

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Trip Duration,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,1.55,6.508813,3.377403,6.650970,3.345031
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,29.40,6.431671,3.455538,6.428081,3.472189


In [15]:
#completed_orders.drop(columns=['Trip Origin', 'Trip Destination'], inplace=True)
completed_orders

,Trip ID,Trip Start Time,Trip End Time,Trip Duration,Origin Latitude,Origin Longitude,Destination Latitude,Destination Longitude
0,391996,2021-07-01 07:28:04,2021-07-01 07:29:37,1.550000,6.508813,3.377403,6.650970,3.345031
1,391997,2021-07-01 06:38:04,2021-07-01 07:07:28,29.400000,6.431671,3.455538,6.428081,3.472189
2,391998,2021-07-01 06:21:02,2021-07-01 07:02:23,41.350000,6.631679,3.338898,6.508324,3.359040
3,391999,2021-07-01 07:16:07,2021-07-01 07:29:42,13.583333,6.572757,3.367708,6.584881,3.361407
4,392001,2021-07-01 09:30:59,2021-07-01 09:34:36,3.616667,6.601042,3.276634,6.450107,3.391615
...,...,...,...,...,...,...,...,...
534363,1637696,2021-12-30 20:35:06,2021-12-30 21:02:59,27.883333,6.448218,3.477208,6.437787,3.481670
534364,1637702,2021-12-30 20:48:13,2021-12-30 21:43:49,55.600000,6.442321,3.473687,6.436589,3.555974
534365,1637704,2021-12-30 20:51:45,2021-12-30 21:41:32,49.783333,6.428198,3.492248,6.448089,3.477575
534366,1637705,2021-12-30 20:48:50,2021-12-30 21:08:28,19.633333,6.586930,3.363297,6.637907,3.333951


In [23]:
from geopy.distance import geodesic
import haversine as hs
import pandas as pd

# Helper function to compute geodesic distance
def compute_geodesic_distance(row):
    start_coords = (row['Origin Latitude'], row['Origin Longitude'])
    end_coords = (row['Destination Latitude'], row['Destination Longitude'])
    return geodesic(start_coords, end_coords).meters

# Helper function to compute haversine distance
def compute_haversine_distance(row):
    start_coords = (row['Origin Latitude'], row['Origin Longitude'])
    end_coords = (row['Destination Latitude'], row['Destination Longitude'])
    return hs.haversine(start_coords, end_coords) * 1000  # Convert to meters

# Helper function to compute speed
def compute_speed(row):
    distance = row['Geodesic Distance (m)']  # You can choose geodesic or haversine distance here
    time_diff = (row['Trip End Time'] - row['Trip Start Time']).total_seconds() / 3600.0  # in hours
    if time_diff == 0:
        return 0
    return distance / time_diff


In [24]:
completed_orders['Geodesic Distance (m)'] = completed_orders.apply(compute_geodesic_distance, axis=1)



In [ ]:
completed_orders['Haversine Distance (m)'] = completed_orders.apply(compute_haversine_distance, axis=1)


In [ ]:
completed_orders['Speed (m/s)'] = completed_orders.apply(compute_speed, axis=1)